In [29]:
import logging
import os
import pymongo
import cobrakbase
import time

logger = logging.getLogger(__name__)
%run ../../scripts/bios_utils.py

In [2]:
kbase = cobrakbase.KBaseAPI(token='UGOG6KLAWTCYI2ASYECYHNIIFTEXGA2J')

In [31]:
ref_info = kbase.get_object_info_from_ref('49164/7/1')
print(ref_info, ref_info.id, ref_info.workspace_id)

49164/7/1 FungalOrthos janakakbase:narrative_1570052138482


In [7]:
fungal_orthos = kbase.get_object(ref_info.id, ref_info.workspace_id)

In [ ]:
write_json(fungal_orthos, '../../../data/kbase_data/janakakbase_narrative_1570052138482/')

In [10]:
for k in fungal_orthos.keys():
    print(k, type(fungal_orthos[k]))

genome_refs <class 'list'>
id <class 'str'>
name <class 'str'>
orthologs <class 'list'>
type <class 'str'>


In [20]:
#GCF_000277815.2 Encephalitozoon_hellem_ATCC_50504
#GCF_000143185.1 Schizophyllum_commune_H4-8
for genome_ref in fungal_orthos['genome_refs']:
    ref_info = kbase.get_object_info_from_ref(genome_ref)
    print(ref_info.id, genome_ref, ref_info.workspace_id)

GCF_000226395.1 49164/6/1 janakakbase:narrative_1570052138482
GCF_000182925.2 49164/5/1 janakakbase:narrative_1570052138482
GCF_000146045.2 49164/4/1 janakakbase:narrative_1570052138482
Encephalitozoon_hellem_ATCC_50504 49164/3/1 janakakbase:narrative_1570052138482
Schizophyllum_commune_H4-8 49164/2/1 janakakbase:narrative_1570052138482


In [28]:
for o in fungal_orthos['orthologs']:
    if o['id'] == 'cluster844':
        print(o['id'], o['function'])
        for ortholog in o['orthologs']:
            print(ortholog)
        break

cluster844 (R,R)-butanediol dehydrogenase
['Pc23g00940', 13504, '49164/6/1']
['Pc21g01400', 139, '49164/6/1']
['Pc21g02320', 234, '49164/6/1']
['YAL061W', 10, '49164/4/1']
['YAL060W', 11, '49164/4/1']


In [21]:
genome_data = kbase.get_object('GCF_000146045.2', 'janakakbase:narrative_1570052138482')

In [27]:
genome_data['features'][4955]['id']

'YNL284C-B'